In [ ]:
import pandas as pd
import os
from datetime import datetime
import mydashtools as dt
import plotly.express as px
pd.options.plotting.backend = "plotly"

services_baristas = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../calendar/baristas.csv", sep=";",decimal=',',low_memory=False, index_col=False)
services_baristas['mois'] = pd.to_datetime(services_baristas['mois'])
if dt.is_notebook():
    print(services_baristas.head())

In [ ]:
creneaux_annules = services_baristas[services_baristas['role'] == "CRENEAU ANNULE"][['mois', 'role']].groupby('mois').count()
creneaux_annules['year'] = creneaux_annules.index.year
creneaux_annules = creneaux_annules.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)
creneaux_annules.columns = creneaux_annules.columns.droplevel(0)

fig = px.bar(creneaux_annules)
dt.fig_save_or_show(fig, 'barista_01_creneaux_annules')

In [ ]:
nb_baristas = services_baristas[services_baristas['role'] != "CRENEAU ANNULE"].groupby(['mois','nom','telephone']).count().rename(columns= {'role': 'nb'})
nb_baristas = nb_baristas.groupby('mois').aggregate(func=['count', 'sum'])
nb_baristas.columns = nb_baristas.columns.droplevel(0)
nb_baristas = nb_baristas.rename(columns={'count': 'nb baristas', 'sum': 'nb services réalisés'})
nb_baristas['year'] = nb_baristas.index.year
nb_baristas = nb_baristas.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)

In [ ]:
fig = px.bar(nb_baristas[['nb baristas']].stack('year').groupby('year').sum())
dt.fig_save_or_show(fig, 'barista_02_nb_baristas_par_an')

In [ ]:
fig = px.bar(nb_baristas[['nb services réalisés']].stack('year').groupby('year').sum())
dt.fig_save_or_show(fig, 'barista_03_nb_services_realises_par_an')

In [ ]:
fig = px.bar(nb_baristas['nb baristas'])
dt.fig_save_or_show(fig, 'barista_04_nb_baristas_par_mois')

In [ ]:
fig = px.bar(nb_baristas['nb services réalisés'])
dt.fig_save_or_show(fig, 'barista_05_nb_services_réalisés_par_mois')

In [ ]:
services = services_baristas[services_baristas['role'] == "REFERENT"].groupby(['mois','nom','telephone']).count().rename(columns= {'role': 'nb'})
services = services.groupby('mois').aggregate(func=['count', 'sum'])
services.columns = services.columns.droplevel(0)
services['year'] = services.index.year

services = services.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)

In [ ]:
fig = px.bar(services['count'])
dt.fig_save_or_show(fig, 'barista_06_nb_referents_par_mois')

In [ ]:
fig = px.bar(services['sum'])
dt.fig_save_or_show(fig, 'barista_07_nb_services_ouverts')

In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
baristas_t2n = services_baristas.groupby(['telephone'])['nom','role'].first()
baristas = services_baristas[services_baristas['mois'] > limit_date][services_baristas['role'] != "CRENEAU ANNULE"].groupby(['telephone'])[['role']].count()
baristas['nb'] = services_baristas[services_baristas['mois'] > limit_date][services_baristas['role'] != "CRENEAU ANNULE"].groupby(['telephone'])[['role']].count()
baristas.sort_values(by='nb', ascending=False, inplace=True)
baristas['nom'] = baristas.index.map(baristas_t2n['nom'])
baristas = baristas.reset_index()[['nom', 'nb']].groupby('nom')[['nb']].sum().sort_values(by='nb', ascending=False)

fig = px.bar(baristas)
dt.fig_save_or_show(fig, 'barista_08_nb_services_par_baristas_12_mois')

In [ ]:
if dt.is_notebook():
    print(baristas.head(30))

In [ ]:
res={
    'un_seul_service': len(baristas[baristas[['nb']] < 2].dropna().reset_index()['nom'].unique()),
    'plus_de_20_services': len(baristas[baristas[['nb']] > 20].dropna().reset_index()['nom'].unique()),
    'plus_de_40_services': len(baristas[baristas[['nb']] > 40].dropna().reset_index()['nom'].unique()),
    'plus_de_60_services': len(baristas[baristas[['nb']] > 60].dropna().reset_index()['nom'].unique()),
    'nb_baristas': len(baristas.dropna().reset_index()['nom'].unique()),
    'moyenne_du_nb_de_services': baristas['nb'].mean()
}
barista_std = baristas.reset_index()
res['ecart_type_du_nb_de_services'] = barista_std[barista_std.index == len(barista_std)/2].reset_index()['nb'][0]
if dt.is_notebook():
    print(res)